### Program written by Scott Midgley, 2021
Scope: To ingest VASP energies from .csv format and generate Coulomb matrix eigenspectrum from POSCAR structure files. Output              saved as .pkl file, ready for machine learning models.

In [1]:
### USER INPUT REQUIRED ###

# Please paste in the path to the repositiory here an comment/uncomment as needed.
# E.g. rundir = r'C:\Users\<user>\Desktop\repository'

# Windows path
#repodir = r'<windows\path\here>'

#Unix path
#repodir = '<unix/path/here>'
repodir = r'/home/mts87985/ml-thermo/Machine-Learning-for-Solid-Solutions/repository/'

In [2]:
# Import modules.
import pandas as pd
import os
import numpy as np
from pymatgen import Structure
from matminer.featurizers import structure as sf
import time

In [3]:
# Start program timer.
start_time = time.time()

In [4]:
# Read DFT derived energies from .csv file to data frame.
energies = pd.read_csv(repodir + "data/vasp-energies.csv", header=None)
energies.columns = ['SCF Energy', 'BGE']

In [30]:
# Iterate over structures in structure directory, generating CME for each configuration.
cm_list = []
files = [f for f in os.listdir(repodir + '/data/structure_files')]

In [32]:
for f in files:
     struct = Structure.from_file(repodir + 'data/structure_files/' + f)
     cm = sf.CoulombMatrix()
     fitted_matrix = cm.fit([struct])
     featurized_structure = fitted_matrix.featurize(struct)
     cm_list.append(np.sort(featurized_structure)[::-1])
print('Number of matrices read: ', len(cm_list))

Number of matrices read:  8043


In [33]:
# Add CME's to data frame with DFT energies. 
ener = energies.iloc[:len(cm_list)]
ener["Coulomb"] = cm_list

In [34]:
# Shuffle data frame (optional).
ener = ener.sample(frac=1)

In [35]:
len(files)

8043

In [ ]:
# Save data frame to .pkl file.
ener.to_pickle('input_data.pkl')

In [ ]:
# Print time taken by program to run. 
time_s = round((time.time() - start_time), 2)
time_m = round((time_s/60), 2)
print(time_s,'sec')
print(time_m, 'min')